In [ ]:
"""
import packages
"""
import os, glob
import json
import pandas as pd
import numpy as np
import math, random
from datetime import datetime
import pickle
from matplotlib import pyplot as plt
import argparse
import path, sys, re, time
from collections import Counter
from scipy.spatial import distance_matrix

In [ ]:
"""
import custom packages
"""
from module_.info.testbed_info import activityfiles_new
from module_.info.config import config, feature_name
from module_.readText import create_episodes, time_correction
from module_.featureExtraction import feature_extraction
from module_.changePointDetection import change_point_detection

In [ ]:
"""
load raw files
"""
dir_="dataset/testbed/npy/seminar/MS"
task_dict={i:[np.load("{}/{}".format(dir_, name)) for name in v] for i, v in enumerate(activityfiles_new.values())}
initial_dict={i:k[0] for i, k in enumerate(activityfiles_new.keys())}
label_dict={k[0]:k for k in activityfiles_new.keys()}

episodes, trs, tags = create_episodes(task_dict, initial_dict)
episodes=[time_correction(eps, trs[i]) for i, eps in enumerate(episodes)]

In [ ]:
dataset='testbed'
preprocess='MS'
metric='RuLSIF'

window_size=30
threshold=120

bigbucket=[]

for pairfolder in glob.glob("./outputs/{}/{}/*".format(dataset, preprocess)):
    print(pairfolder.split("/")[-1])
    for epsfolder in glob.glob("{}/*".format(pairfolder)):
        
        epsorder, name_, changeidx = epsfolder.split("/")[-1].split("_")

        events=np.load("{}/events.npy".format(epsfolder))
        features=np.load("{}/features.npy".format(epsfolder))
        scores=np.load("{}/{}/scores.npy".format(epsfolder, metric))

        sensor_list=sorted(set(events[:,0]))

        # file_=open("{}/report_wo_c.txt".format(epsfolder), 'w')
        # file_.write('{} Transition at {}\n'.format(name_, changeidx))
        # file_.write('{} {}\n'.format(sensor_list, len(sensor_list)))

        event_file=open("{}/eventA.txt".format(epsfolder), 'w')

        timestamps=list(events[:,2].astype(float))
        transitionTimestamp=timestamps[int(changeidx)]

        # # find index
        timeA=[t for t in timestamps if t-timestamps[0]<threshold][-1]
        timeB=[t for t in timestamps if transitionTimestamp-t<threshold][0]
        timeC=[t for t in timestamps if t-transitionTimestamp<threshold][-1]
        timeD=[t for t in timestamps if timestamps[-1]-t<threshold][0]

        idxA=timestamps.index(timeA)
        idxB=timestamps.index(timeB)
        idxC=timestamps.index(timeC)
        idxD=timestamps.index(timeD)

        # # event
        eventA=events[:idxA+1]
        eventB=events[idxB-window_size+1:int(changeidx)]
        eventC=events[int(changeidx)-window_size+1:idxC+1]
        eventD=events[idxD-window_size+1:]

        event_file.write("First Start\n")
        for i in range(len(eventA)):
            event_file.write("{}\n".format(eventA[i]))
        event_file.write("First End\n")
        for i in range(len(eventB)):
            if i==window_size-1:
                event_file.write("{} *\n".format(eventB[i]))
            else:
                event_file.write("{}\n".format(eventB[i]))
        event_file.write("Second Start\n")
        for i in range(len(eventC)):
            if i==window_size-1:
                event_file.write("{} *\n".format(eventC[i]))
            else:
                event_file.write("{}\n".format(eventC[i]))
        event_file.write("Second End\n")
        for i in range(len(eventD)):
            if i==window_size-1:
                event_file.write("{} *\n".format(eventD[i]))
            else:
                event_file.write("{}\n".format(eventD[i]))
        
        event_file.close()

        # # feature
        # featureA=features[:idxA+1]
        # featureB=features[idxB:int(changeidx)]
        # featureC=features[int(changeidx):idxC+1]
        # featureD=features[idxD:]

        # # score
        # scoreA=scores[:idxA+1]
        # scoreB=scores[idxB:int(changeidx)]
        # scoreC=scores[int(changeidx):idxC+1]
        # scoreD=scores[idxD:]

        # lambdas=np.load("{}/{}/lambdas.npy".format(epsfolder, metric))
        # sigmas=np.load("{}/{}/sigmas.npy".format(epsfolder, metric))
        # thetas=np.load("{}/{}/thetas.npy".format(epsfolder, metric))

        # descending=sorted([(score, idx) for idx, score in enumerate(scores)], reverse=True)

        # for i in range(100):
        #     order=i
        #     score__, eidx=descending[order]
        #     # print("{} Score at {}".format(score__, eidx))
        #     file_.write("{} Score at {}\n".format(score__, eidx))
        #     file_.write("{}\n{}\n{}\n".format(events[i], events[min(len(events)-1, i+1)], events[min(len(events)-1, i+2)]))
        #     e_=events[max(0, eidx-30+1):eidx+3]
        #     # f_=features[eidx:eidx+3]

        #     f1, f2, f3=features[eidx:eidx+3]
            
        #     f1f2=np.array(f1)-np.array(f2)
        #     dict_={}
        #     total=0
        #     for i in range(len(f1f2)):
        #         dict_[i]=f1f2[i]**2
        #         total+=f1f2[i]**2
        #     dict__={}
        #     for i in range(len(f1f2)):
        #         value=round(dict_[i]/total,2)
        #         if value>0.1:
        #             dict__[i]=value
        #     importances_f1f2=sorted(dict__.items(), key=lambda x: x[1], reverse=True)

        #     f1f3=np.array(f1)-np.array(f3)
        #     dict_={}
        #     total=0
        #     for i in range(len(f1f3)):
        #         dict_[i]=f1f3[i]**2
        #         total+=f1f3[i]**2
        #     dict__={}
        #     for i in range(len(f1f3)):
        #         value=round(dict_[i]/total,2)
        #         if value>0.1:
        #             dict__[i]=value
        #     importances_f1f3=sorted(dict__.items(), key=lambda x: x[1], reverse=True)

        #     f2f3=np.array(f2)-np.array(f3)
        #     dict_={}
        #     total=0
        #     for i in range(len(f2f3)):
        #         dict_[i]=f2f3[i]**2
        #         total+=f2f3[i]**2
        #     dict__={}
        #     for i in range(len(f2f3)):
        #         value=round(dict_[i]/total,2)
        #         if value>0.1:
        #             dict__[i]=value
        #     importances_f2f3=sorted(dict__.items(), key=lambda x: x[1], reverse=True)

        #     L2_12=sum(np.square(f1f2))
        #     L2_13=sum(np.square(f1f3))
        #     L2_23=sum(np.square(f2f3))

        #     # print("1-2", L2_12)
        #     # print("1-3",L2_13)
        #     # print("2-3", L2_23)
        #     file_.write("1-2 {}\n".format(L2_12))
        #     file_.write("1-3 {}\n".format(L2_13))
        #     file_.write("2-3 {}\n".format(L2_23))

        #     # print("1-2 / 2-3", L2_12/L2_23)
        #     # print("1-3 / 1-2", L2_13/L2_12)
        #     file_.write("1-2 / 2-3 {}\n".format(L2_12/L2_23))
        #     file_.write("1-3 / 1-2 {}\n".format(L2_13/L2_12))

        #     file_.write("f1 and f2 {}\n".format(importances_f1f2))
        #     file_.write("f1 and f3 {}\n".format(importances_f1f3))
        #     file_.write("f2 and f3 {}\n".format(importances_f2f3))
        #     # print("f1 and f2", importances_f1f2)
        #     # print("f1 and f3", importances_f1f3)
        #     # print("f2 and f3", importances_f2f3)

        # # print(e_, len(e_))
        # file_.close()

        # for index_ in range(len(events)):

        #     score=scores[index_]
        #     theta=thetas[index_]
        #     lambda_, sigma_ = lambdas[index_], sigmas[index_,0]
        #     # print(score, lambda_, sigma_)
        #     # print(theta)

        #     before=np.array([features[index_], features[min(index_+1, len(events)-1)]])
        #     after=np.array([features[min(index_+1, len(events)-1)], features[min(index_+2, len(events)-1)]])

        #     L2_test=np.square(distance_matrix(before, before))
        #     L2_train=np.square(distance_matrix(before, after))

        #     phi_test=np.exp(-L2_test/(2*(sigma_**2)))
        #     phi_train=np.exp(-L2_train/(2*(sigma_**2)))

        #     # print(phi_test)
        #     # print(phi_train)

        #     g_x=phi_test.T@theta
        #     g_y=phi_train.T@theta

        #     # print(g_x)
        #     # print(g_y)

        #     # for alpha in range(0, 10, 1):
        #     #     alpha=float(alpha/10.)
        #     #     estimated_score=-0.5-alpha*0.5*np.square(g_x).mean()-(1-alpha)*0.5*np.square(g_y).mean()+g_x.mean()
        #     #     print(estimated_score)
        #     bucket=[]
        #     bucket.append(score)
        #     bucket+=list(g_x.ravel())
        #     bucket+=list(g_y.ravel())
        #     bucket+=list(theta.ravel())
        #     bucket+=[lambda_, sigma_]
        #     bucket+=list(L2_test.ravel())
        #     bucket+=list(L2_train.ravel())
        #     bucket+=list(phi_test.ravel())
        #     bucket+=list(phi_train.ravel())


        #     bigbucket.append(bucket)

# np.savetxt("analysis.csv", bigbucket, delimiter=',')